In [8]:
import tensorflow as tf
from keras_squeeze_excite_network import se
from tensorflow import keras
from tensorflow.keras import activations, layers

In [ ]:
ResidualTower = None
SqueezeAndExite = None

In [15]:
# == BODY == #
FILTERS = 128
inputs = layers.Input(shape=(8, 8, 6))
y = layers.Conv2D(FILTERS, (3, 3), padding="same")(inputs)

# == RESIDUAL_TOWER == #
BLOCKS = 10
for _ in range(BLOCKS):
    x = layers.Conv2D(FILTERS, (3, 3), padding="same")(y)
    x = layers.Conv2D(FILTERS, (3, 3), padding="same")(x)

    # == SQUEEZE_AND_EXITATION == #
    SE_CHANNELS = 32
    x = layers.GlobalAveragePooling2D()(y)
    x = layers.Dense(SE_CHANNELS, activation="relu")(x)
    x = layers.Reshape((2, FILTERS))(x)
    x = layers.Dense(FILTERS)(x)
    W, B = tf.unstack(x, axis=1)
    Z = layers.Activation("sigmoid")(W)
    # Use lambda here
    x = layers.Add()((layers.Multiply()((Z, y)), B))

    x = layers.Add()((x, y))
    x = layers.ReLU()(x)

# == POLICY_HEAD == #
POLICY_CONV_SIZE = 32
ph = layers.Conv2D(POLICY_CONV_SIZE, (3, 3), padding="same")(x)
ph = layers.Flatten()(ph)
ph = layers.Dense(1858)(ph)

# == VALUE_HEAD == #
vh = layers.Conv2D(32, (3, 3), padding="same")(x)
vh = layers.Conv2D(128, (3, 3), activation="relu")(x)
ph = layers.Flatten()(x)
vh = layers.Dense(1, activation="tanh")(vh)

In [3]:
ph.shape, vh.shape

(TensorShape([None, 8192]), TensorShape([None, 6, 6, 1]))

In [16]:
model = keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 8, 8, 6)]    0                                            
__________________________________________________________________________________________________
conv2d_60 (Conv2D)              (None, 8, 8, 128)    7040        input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 8, 8, 128)    147584      conv2d_60[0][0]                  
__________________________________________________________________________________________________
lambda_13 (Lambda)              (None, 8, 8, 128)    0           conv2d_61[0][0]                  
____________________________________________________________________________________________

In [21]:
import numpy as np
from matplotlib import pyplot as plt

f = np.load("../data/npz/ficsgamesdb_2015_standard2000_nomovetimes.npz")

X, y = f["X"], f["y"]

print(y.mean(), y.var())
len(y[abs(y) < 0.1])
# len(y)

0.1487505842230979 0.4911248313238087


7686